In [1]:
import pydrake
from pydrake.all import (
    AngleAxis_,
    AutoDiffXd,
    Expression,
    MathematicalProgram,
    Quaternion_,
    RotationMatrix_,
    RotationMatrix,
    RollPitchYaw_,
    RollPitchYaw,
    Solve,
    Variable
)
import pydrake.math as dmath
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = np.loadtxt("../calibration_pairs.csv", delimiter=",")
print("Loaded %d points " % data.shape[0])

d415_intrinsics = np.loadtxt("../d415_intrinsics.csv")

# 2xN points in projector image plane
uv_projected = data[:, :2].T
print(np.max(uv_projected, axis=1))
# 2xN points in RGBD image plane (unused)
uv_observed = data[:, 2:4].T
print(np.max(uv_observed, axis=1))
# 3xN points in RGBD camera frame
p_c = data[:, 4:].T
print(np.min(p_c, axis=1), np.max(p_c, axis=1))
# recalculate... bug?
#p_c[:2, :] = np.linalg.inv(d415_intrinsics)[:2, :2].dot(uv_observed) * p_c[2, :]

Loaded 1096 points 
[1267.875     758.931335]
[522.224731 396.973724]
[-0.794943 -0.4857    0.728   ] [0.835286 0.531978 3.286   ]


In [3]:
def get_R(R_rpy):
    if isinstance(R_rpy[0], AutoDiffXd):
        return RotationMatrix_[AutoDiffXd](RollPitchYaw_[AutoDiffXd](R_rpy))
    elif isinstance(R_rpy[0], (Expression, Variable)):
        return RotationMatrix_[Expression](RollPitchYaw_[Expression](R_rpy))
    else:
        return RotationMatrix(RollPitchYaw(R_rpy))
def get_K(fov_xy, c_xy):
    # tan(fov / 2) = (resolution / 2) / f_x
    f_x = (native_resolution[0] / 2.) / dmath.tan(fov_xy[0] / 2)
    f_y = (native_resolution[1] / 2.) / dmath.tan(fov_xy[1] / 2)
    return np.array([[f_x, 0., c_xy[0]],
                  [0., f_y, c_xy[1]],
                  [0., 0., 1.]])
fov_xy_guess = 2*np.arctan2(np.array([20., 13.5])/2., 26)
native_resolution = np.array([1280, 1024])
c_xy_guess = native_resolution / 2

In [4]:
# From empirical measurements
print("Guess: ", fov_xy_guess, c_xy_guess)


def setup_and_solve():
    prog = MathematicalProgram()
    # We're trying to find the pose of the projector,
    # and its FOV. (We know its full resolution.)

    # Set up extrinsics variables
    T = prog.NewContinuousVariables(3, "T")
    R_rpy = prog.NewContinuousVariables(3, "R_rpy")
    prog.AddBoundingBoxConstraint(np.ones(3)*-0.5, np.ones(3)*0.5, T)
    prog.AddBoundingBoxConstraint(np.ones(3)*-10.*np.pi, np.ones(3)*10.*np.pi, R_rpy)
    prog.SetInitialGuess(T, np.random.random(3)*0.1 - 0.05)
    prog.SetInitialGuess(R_rpy, [0., 0., 0.])


    # Set up intrinsics variables
    fov_xy = prog.NewContinuousVariables(2, "fov_xy")
    prog.AddBoundingBoxConstraint(fov_xy_guess * 0.5, fov_xy_guess*2.0, fov_xy)
    prog.SetInitialGuess(fov_xy, fov_xy_guess)
    
    c_xy = prog.NewContinuousVariables(2, "c_xy")
    prog.AddBoundingBoxConstraint(c_xy_guess - 100, c_xy_guess + 100, c_xy)
    prog.SetInitialGuess(c_xy, c_xy_guess)
    

    # Trivial constraint: transformed points must still be in front of camera,
    # which is -z in opengl.
    R = get_R(R_rpy)
    tf_pts = (R.multiply(p_c).T + T).T
    for k in range(p_c.shape[1]):
        prog.AddConstraint(tf_pts[2, k] >= 0.0)
    
    def calc_error(x):
        T = x[:3]
        R_rpy = x[3:6]
        fov_xy = x[6:8]
        c_xy = x[8:10]
        R = get_R(R_rpy)
        K = get_K(fov_xy, c_xy)

        # Project the point set into the camera.
        p_projector = (R.multiply(p_c).T + T).T
        p_uv_projector = K.dot(p_projector)
        p_uv_projector[0, :] /= p_uv_projector[2, :]
        p_uv_projector[1, :] /= p_uv_projector[2, :]
        return np.mean((p_uv_projector[:2, :] - uv_projected)**2.)

    prog.AddCost(calc_error, np.concatenate([T, R_rpy, fov_xy, c_xy]))
    result = Solve(prog)
    
    success = result.is_success()
    T = result.GetSolution(T)
    R_rpy = result.GetSolution(R_rpy)
    fov_xy = result.GetSolution(fov_xy)
    c_xy = result.GetSolution(c_xy)
    final_err = calc_error(np.concatenate([T, R_rpy, fov_xy, c_xy]))
    print("K: ", get_K(fov_xy, c_xy))
    return success, final_err, (T, R_rpy, fov_xy, c_xy)

for k in range(3):
    sol = setup_and_solve()
    print(sol)

Guess:  [0.73434767 0.50801553] [640. 512.]
K:  [[1.70972298e+03 0.00000000e+00 6.36022598e+02]
 [0.00000000e+00 1.82296612e+03 4.12000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(True, 27.145824072303935, (array([-0.05358709,  0.0064919 , -0.04613712]), array([2.22251110e-02, 4.35832961e-04, 3.13568127e+00]), array([0.71636567, 0.54761377]), array([636.0225977, 412.       ])))
K:  [[1.70972298e+03 0.00000000e+00 6.36022598e+02]
 [0.00000000e+00 1.82296612e+03 4.12000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(True, 27.14582407230396, (array([-0.05358709,  0.0064919 , -0.04613712]), array([2.22251110e-02, 4.35832988e-04, 3.13568127e+00]), array([0.71636567, 0.54761377]), array([636.02259775, 412.        ])))
K:  [[1.70972298e+03 0.00000000e+00 6.36022598e+02]
 [0.00000000e+00 1.82296612e+03 4.12000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(True, 27.145824072303927, (array([-0.05358709,  0.0064919 , -0.04613712]), array([2.22251110e-02, 4.358

In [5]:
T, R_rpy, fov_xy = sol[-1]
K = get_K(fov_xy)
R = get_R(R_rpy)
tf = np.eye(4)
tf[:3, :3] = R.matrix()
tf[:3, 3] = T
print(tf)
print(K)

ValueError: too many values to unpack (expected 3)

In [ ]:
import meshcat
import meshcat.geometry as g
vis = meshcat.Visualizer(zmq_url="tcp://127.0.0.1:6000")
vis.delete()
vis['p_c'].set_object(
    g.PointsGeometry(position=p_c),
    g.PointsMaterial(size=0.05, color=0xff0000))

proj_p = K.dot((R.multiply(p_c).T + T).T)
proj_p[0, :] /= proj_p[2, :] * 1000.
proj_p[1, :] /= proj_p[2, :] * 1000.
proj_p[2, :] /= proj_p[2, :]
print(proj_p)
vis["p_uv"].set_object(
    g.PointsGeometry(position=proj_p),
    g.PointsMaterial(size=0.05, color=0x0000ff))
uv_pts = np.vstack([uv_projected, np.ones((1, uv_projected.shape[1]))])
vis['uv'].set_object(
    g.PointsGeometry(uv_pts/1000.),
    g.PointsMaterial(size=0.05, color=0x00ff00))

paired = np.empty((3, 2*p_c.shape[1]))
paired[:, ::2] = p_c
paired[:, 1::2] = uv_pts/1000.
vis['corr'].set_object(
    g.LineSegments(g.PointsGeometry(paired)))